# Исследование надежности заемщиков.

### Задача проекта:
Разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Заказчик — кредитный отдел банка. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Обзор данных

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
import os

In [2]:
#прочитаем датасет
pth1 = '/datasets/data.csv'
pth2 = 'C:/Users/Olga/Desktop/Проект Банк/data.csv'
if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
else:
    print('something is wrong')    
#посмотрим информацию по датасету
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# посмотрим на данные в таблице
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
# посмотрим на данные в таблице
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Вывод: Есть одинакове количество пропусков в столбцах days_employed и total_income. Так как их количесство одинаково в обох столбцах, то можно предполодить, что человек не был трудоустроен официально. 
В столбце days_employed вещественные данные, чего не может быть, также есть отрицательные значение, чего также быть не может. 
В стообце education использованы разные регистры для написания, необходимо привести столбец к одному виду.
В столбце total_income представлены вещественные данные, лучше заменить на целочисленные, так как зп в месяц в основном целое чило. 

In [5]:
#проверка написания столбцов, ошибок не воявлено
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [6]:
#приведем к одноу регистру столбец education.
data['education'] = data['education'].str.lower()

### Шаг 2.1 Заполнение пропусков

In [7]:
#посмотрим, в какой столбцах есть пропуски и какое количество.
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [8]:
#посмотрим, что представляют из себя пустые значения.
data[data['total_income'].isna()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [9]:
#сводная таблица поможет проверить корректность данных(медианы), подставляемых функцией.
data_pivot = data.pivot_table(index=['education', 'income_type'], values='total_income', aggfunc='median')
data_pivot.reset_index().head()


,education,income_type,total_income
0,высшее,безработный,202722.511368
1,высшее,госслужащий,172511.107016
2,высшее,компаньон,201785.400018
3,высшее,пенсионер,144240.768611
4,высшее,предприниматель,499163.144947


In [10]:
#столбец total_income можно заполнить медианными значениями в зависимости от income_type и от education. 
#вариант с медианой наиболее выйграшный, так как считает средний доход, а не среднее между минимальным и максимальным значением. 
#функция считает медианное значение по уровню образования и типу занятости
def mediana_by_education_and_income(data, education, income_type):
    mediana = data[(data['education'] == education) & (data['income_type'] == income_type)]['total_income'].median()
    return mediana
#проверка работы функции(полученное значение сравниваю со значением из сводной таблицы)
med = mediana_by_education_and_income(data, 'высшее', 'пенсионер')
med


144240.76861073746

In [11]:
#в цикле проходимся по одразованию и типу занятости и в пустые ячейки ежемесячного дохода(total_income) Подставляем медианное значение
for education in data[data['total_income'].isna()]['education']:
    for income_type in data[data['total_income'].isna()]['income_type']:
        med = mediana_by_education_and_income(data, education, income_type)
        row = (data['education'] == education) & (data['income_type'] == income_type) & (data['total_income'].isna())
        data.loc[row, 'total_income'] = data.loc[row, 'total_income'].fillna(med)
#проверка, заполнились ли пустые значения в столбце ежемесячного дохода(total_income) 
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [12]:
#посмотрм, что представляют из себя значения в столбце days_employed
data['days_employed'].value_counts()

-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64

In [13]:
#т.к. есть отрицательные значения,скорее всего это опечатка, ведь невозможно,
#чтобы количество отработанных дней было отрицательным числом, заменим их положителными.
data['days_employed'] = data['days_employed'].abs()
#найдем медиану и заменим пустые значения медианой. Пустые значения могут означать, что человек работает не официально.
med = data['days_employed'].median()
data.loc[data['days_employed'].isna(), 'days_employed'] = med
#проверим, остались ли пропуски, вызвав метод info()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [14]:
#количество отработанных дней days_employed лучше отображать целым числом, так как обычно учет ведется в целых днях или сменах. 
data['days_employed'] = data['days_employed'].astype(int)
data['days_employed'].value_counts()

2194      2180
133         16
327         16
438         15
223         14
          ... 
8200         1
9090         1
360849       1
2101         1
343937       1
Name: days_employed, Length: 9086, dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [15]:
#посмотрим, какие данные есть в столбце gender.
data['gender'].value_counts().to_frame()

,gender
F,14236
M,7288
XNA,1


In [16]:
#в одном случае гендер не определен, удалим его, чтобы остались только изветсные значения. 
data['gender'] = data.loc[data['gender'] != 'XNA', 'gender']
data['gender'].value_counts().to_frame()

,gender
F,14236
M,7288


In [17]:
#посмотрим, какие данные есть в столбце children.
data['children'].value_counts().to_frame()

,children
0,14149
1,4818
2,2055
3,330
20,76
-1,47
4,41
5,9


In [18]:
#т.к.отрицательное число детей быть не может, проверим семейный статус тех, у кого вышло -1.
data.loc[data['children'] == -1]['family_status'].value_counts()

женат / замужем          29
гражданский брак          5
Не женат / не замужем     5
в разводе                 4
вдовец / вдова            4
Name: family_status, dtype: int64

In [19]:
#так как большинство были в браке или соcтоят в нем, то предположим, что опечатка в знаке. Заменим значение на положительное.
data.loc[data['children'] == -1, 'children'] = 1

#также число 20 детей кажется ошибочным, скорре всего опечатка, добавили лишний 0, исправим.
data.loc[data['children'] == 20, 'children'] = 2

data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [20]:
#посмотрим, нет ли каких то аномальных значений
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

In [21]:
#найдем медиану, чтобы заменить 0 возраст заемщика, так как он не может быть 0
#data_dod_avr = data['dob_years'].median()
#data_dod_avr


In [22]:
#заменим 0 медианным значением
#data.loc[data['dob_years'] == 0, 'dob_years'] = data_dod_avr
#data['dob_years'].unique().astype(int)

In [23]:
#приведем цели на кредит к одной форме - без пробелов и строчным буквам
data['purpose'].str.strip().str.lower().value_counts().to_frame()    

,purpose
свадьба,797
на проведение свадьбы,777
сыграть свадьбу,774
операции с недвижимостью,676
покупка коммерческой недвижимости,664
покупка жилья для сдачи,653
операции с жильем,653
операции с коммерческой недвижимостью,651
покупка жилья,647
жилье,647


In [24]:
#исправим опечатку "ремонт жилью"
data.loc[data['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'
#проверим, осталась ли опечатка 
data.loc[data['purpose'] == 'ремонт жилью', 'purpose'].count()

0

### Шаг 2.3. Изменение типов данных.

In [25]:
#заменим вещественный тип данных в столбце total_income на целочисленный.
data['total_income'] = data['total_income'].astype(int)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

In [26]:
#проверим количество дубликатов
data.duplicated().sum()

71

In [27]:
#удалим дубликаты методом drop_duplicates
data = data.drop_duplicates(subset=None, keep='first', inplace=False,)
#проверим, остались ли дубликаты
data.duplicated().sum()

0

In [28]:
#приведем к одноу регистру столбец education.
data['education'] = data['education'].str.lower()

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [29]:
#создадим новый датафрейм education_dictionary
education_dictionary = data[['education', 'education_id']]
education_dictionary.drop_duplicates()

,education,education_id
0,высшее,0
1,среднее,1
13,неоконченное высшее,2
31,начальное,3
2963,ученая степень,4


In [30]:
#создадим новый датафрейм family_dictionary
family_dictionary = data[['family_status', 'family_status_id']]
family_dictionary.drop_duplicates()

,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,Не женат / не замужем,4


In [31]:
#удалим из исходного датафрейма столбцы education и family_status
del data['education']
del data['family_status']
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int32 
 2   dob_years         21454 non-null  int64 
 3   education_id      21454 non-null  int64 
 4   family_status_id  21454 non-null  int64 
 5   gender            21453 non-null  object
 6   income_type       21454 non-null  object
 7   debt              21454 non-null  int64 
 8   total_income      21454 non-null  int32 
 9   purpose           21454 non-null  object
dtypes: int32(2), int64(5), object(3)
memory usage: 1.6+ MB


### Шаг 2.6. Категоризация дохода.

In [32]:
#функция, которая будет определять категорию по доходу
def category_by_income(total_income):
    if 0 <= total_income <= 30000:
        return('E')
    elif 30001 <= total_income <= 50000:
        return('D')    
    elif 50001 <= total_income <= 200000:
        return('C') 
    elif 200001 <= total_income <= 1000000:
        return('B') 
    else:
        return('A')
    
category_by_income(145885)

'C'

In [33]:
#проверка работы функции 
data['total_income_category'] = data['total_income'].apply(category_by_income)
data['total_income_category'].value_counts()

C    15829
B     5228
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [34]:
#функция, которая будет определять категорию по цели кредита
def purpose_by_category(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'свадьб'in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
    else: 
        return 'операции с недвижимостью'
        
#проверка работы функции    
purpose_by_category('покупка жилья')
#purpose_by_category('приобретение автомобиля')
#purpose_by_category('сыграть свадьбу')
#purpose_by_category('получение дополнительного образования')


'операции с недвижимостью'

In [35]:
#проверка работы функции
data['purpose_category'] = data['purpose'].apply(purpose_by_category)
data['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [36]:
#построим сводную таблицу 
data_child_debt = data.pivot_table(index=['children'], values='debt', aggfunc = ['sum', 'count'])
#добавим столбец с %, чтобы узнать долю.
data_child_debt['%'] = data_child_debt['sum', 'debt'] / data_child_debt['count', 'debt'] * 100
data_child_debt

,sum,count,%
,debt,debt,
children,,,
0,1063,14091,7.543822
1,445,4855,9.165808
2,202,2128,9.492481
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


Вывод: Если нет детей, то процент задолженности немного ниже, чем когда дети есть. Совсем отсутсвует задолженность, когда детей 5. Но разница в задоженности не совсем значительная.

##### Вопрос 2:Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
#объединим таблицу data с family_dictionary методом merge и по общей таблице построим сводную.
data_family = data.merge(family_dictionary, on='family_status_id', how='left')
data_family_status_debt = data_family.pivot_table(index='family_status', values='debt', aggfunc=['sum', 'count'])
#добавим столбец с %, чтобы узнать долю.
data_family_status_debt['%'] = data_family_status_debt['sum', 'debt'] / data_family_status_debt['count', 'debt'] *100
data_family_status_debt

,sum,count,%
,debt,debt,
family_status,,,
Не женат / не замужем,769940,7896100,9.750890
в разводе,101575,1428025,7.112971
вдовец / вдова,60417,919681,6.569343
гражданский брак,1610588,17230801,9.347145
женат / замужем,11487609,152250921,7.545182


Вывод: Вдовы/вдовцы, находящиеся в разводе и в браке имеют меньший процент задолженности, а не женатые/на замужем, а также в гражданском браке(сожители имеются ввиду, видимо, гражданский, это узаконенный брак) имеют больший процент задолженности.

##### Вопрос 3:Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [38]:
#построим сводную таблицу 
data_income_and_debt = data.pivot_table(index=['total_income_category'],  values='debt', aggfunc=['sum', 'count'])
#добавим столбец с %, чтобы узнать долю.
data_income_and_debt['%'] = data_income_and_debt['sum', 'debt'] / data_income_and_debt['count', 'debt'] * 100
data_income_and_debt

,sum,count,%
,debt,debt,
total_income_category,,,
A,2,25,8.000000
B,364,5228,6.962510
C,1352,15829,8.541285
D,21,350,6.000000
E,2,22,9.090909


Вывод: Люди, относящиеся к категории B(200001–1000000)  и D(30001–50000) имеют меньший процент задолженности, и берут кредиты реже, чем средний класс С, который хуже возвращает кредиты. 

##### Вопрос 4:Как разные цели кредита влияют на его возврат в срок?

In [39]:
#построим сводную таблицу 
data_purpose_and_debt = data.pivot_table(index=['purpose_category'],  values='debt', aggfunc=['sum', 'count'])
#добавим столбец с %, чтобы узнать долю.
data_purpose_and_debt['%'] = data_purpose_and_debt['sum', 'debt'] / data_purpose_and_debt['count', 'debt'] * 100
data_purpose_and_debt

,sum,count,%
,debt,debt,
purpose_category,,,
операции с автомобилем,403,4306,9.359034
операции с недвижимостью,782,10811,7.233373
получение образования,370,4013,9.220035
проведение свадьбы,186,2324,8.003442


Вывод: Больше всего кредит берут на опреции с недвижимостью, и процент возврата по этой категории хороший, так же как и на кредит на проведение свадеб. Хуже обстоят дела с кредитами на автомобиль и образование.

## Общий вывод:

Наиболее подходящий заемщик - у которого нет детей, находящийся в официальных отношениях или разводе, зарабатывайщий выше среднего и берущий кредит на недвижимость.
Но разброс не особо большой и составляет приблизительно 2% в каждом сравнении. Хотя если брать 10% задолженоости за 100%, то разница будет более значительная между должниками от 20-35% 